In [1]:
import os
os.chdir("../")

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BestModelSelectorConfig:
    existing_model_path: Path
    trained_model_path: Path
    score_file_path: Path
    training_data: Path
    params_image_size: list
    params_batch_size: int

In [21]:
from deepClassifier.constants import *
from deepClassifier.utils import *
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_best_model_config(self)->BestModelSelectorConfig:
        best_model_selector = BestModelSelectorConfig(
            existing_model_path=self.config.model_selector.existing_model_path,
            trained_model_path=self.config.model_selector.trained_model_path,
            score_file_path=Path(self.config.model_selector.score_file_path),
            training_data=Path(os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")),
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return best_model_selector     

In [27]:
import tensorflow as tf
import shutil
class BestModelSelector:
    def __init__(self, config: BestModelSelectorConfig):
        self.config = config
        
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        ) 
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        valid_datagenarator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenarator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.exisitng_model_score = None
        if(os.path.exists(self.config.existing_model_path)):
            self.model = self.load_model(self.config.existing_model_path)
            self._valid_generator()
            self.score = self.model.evaluate(self.valid_generator)
            scores = {"loss": self.score, "accuracy": self.score[1]}
            self.exisitng_model_score = ConfigBox(scores)
        self.new_model_score = load_json(self.config.score_file_path)
        
    def select_best_model(self):
        #newly_trained_model_directory = os.path.dirname(self.config.trained_model_path)
        final_destination = os.path.dirname(self.config.existing_model_path)
        if (self.exisitng_model_score == None):
            shutil.copy(self.config.trained_model_path, final_destination)
        else:
            if(self.new_model_score.accuracy > self.exisitng_model_score.accuracy):
                os.remove(self.config.existing_model_path)
                shutil.copy(self.config.trained_model_path, final_destination)
            

In [28]:
try:
    config = ConfigurationManager()
    model_config = config.get_best_model_config()
    selector = BestModelSelector(model_config)
    selector.evaluation()
    selector.select_best_model()
    
except Exception as e:
   raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 1452s 3s/step - loss: 3.7972 - accuracy: 0.7635
